In [1]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import json

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
f = open('/kaggle/input/recipes/recipes.json')
recipes = json.load(f)

In [3]:
for recipe in recipes:
    if ("aggregateRating" not in recipe):
        recipe["aggregateRating"] = {}
        recipe["aggregateRating"]["ratingValue"] = "-"

In [4]:
recipe_info = [
    {
        "name": recipe.get("name", "Recipe Name Not Found"),
        "category": recipe.get("recipeCategory", "Category Not Found"),
        "ingredients": ' '.join(recipe.get("recipeIngredient", [])),  # Convert list to string
        "calories": recipe.get("nutrition", {}).get("calories"),
        "rating": recipe.get("aggregateRating", {}).get("ratingValue"),
        "totalTime": recipe.get("totalTime")
    }
    for recipe in recipes
]

In [5]:
recipe_strings = [' '.join(
    [recipe["category"], 
     recipe["ingredients"],
     recipe["calories"],
     recipe["totalTime"],
     recipe["rating"]]) for recipe in recipe_info]

In [6]:
vectorizer = TfidfVectorizer()
vectorized_data = vectorizer.fit_transform(recipe_strings)

In [7]:
cosine_similarities = cosine_similarity(vectorized_data)

In [8]:
print(cosine_similarities[0])

[1.         0.0290413  0.01037295 ... 0.         0.05587729 0.00661247]


In [9]:
print(cosine_similarities)

[[1.         0.0290413  0.01037295 ... 0.         0.05587729 0.00661247]
 [0.0290413  1.         0.00503506 ... 0.03878908 0.05861981 0.0115287 ]
 [0.01037295 0.00503506 1.         ... 0.10397769 0.01734298 0.12171097]
 ...
 [0.         0.03878908 0.10397769 ... 1.         0.         0.18517438]
 [0.05587729 0.05861981 0.01734298 ... 0.         1.         0.16838974]
 [0.00661247 0.0115287  0.12171097 ... 0.18517438 0.16838974 1.        ]]


In [10]:
def find_similar_recipes(ingredients, category, calories, rating, time, recipe_info, vectorizer, cosine_similarities):
    query_string = ' '.join([category] + ingredients + [calories] + [time] + [rating])
    query_vector = vectorizer.transform([query_string])

    # Calculate cosine similarities between the query vector and all recipe vectors
    similarities = cosine_similarity(query_vector, recipe_info)

    # Get indices of recipes sorted by similarity (descending order)
    similar_recipe_indices = similarities.argsort()[0][::-1]

    return similar_recipe_indices
    

In [11]:
# Example usage:
ingredients_query = ["butter", "pork chops", "apple juice concentrate"]
category_query = "Pork"
calories = "391.7"
rating = "-"
totalTime = "PT25M"
similar_recipes_indices = find_similar_recipes(ingredients_query, category_query, calories, rating, totalTime, vectorized_data, vectorizer, cosine_similarities)

In [12]:
# Print the results
count = 0
number_of_desired_recipes = 10

for idx in similar_recipes_indices:
    count = count + 1
    recipe = recipe_info[idx]
    if count <= number_of_desired_recipes:
        print(f"{recipe['name']} - Category: {recipe['category']}")
        print(f"   Ingredients: {recipe['ingredients']}")
        print(f"   Calories: {recipe['calories']}")
        print(f"   Rating: {recipe['rating']}")
        print(f"   totalTime: {recipe['totalTime']}")
        print()
    
    else:
        break;

Apple Juiced Pork Chops - Category: Pork
   Ingredients: 4       pork chops 2   tablespoons    butter 1   cup    apple juice concentrate
   Calories: 391.7
   Rating: -
   totalTime: PT25M

Orange Pork Chops - Category: Pork
   Ingredients: 4      pork chops   paprika 1  can    orange juice concentrate, thawed  9.85  ml    cinnamon   rice
   Calories: 311.6
   Rating: 4.0
   totalTime: PT1H

Easy Orange Pork - Category: Pork
   Ingredients: 4       pork chops 3   teaspoons   Crisco shortening or 3   teaspoons    margarine 1   can    frozen orange juice concentrate
   Calories: 421.6
   Rating: 5.0
   totalTime: PT30M

Easy Cooked Pork Chops - Category: Pork
   Ingredients: 8       boneless pork loin chops 4   tablespoons    olive oil 2   teaspoons    onions, minced  2 (12   ounce) cans   pork gravy 2   cups    sour cream
   Calories: 183.1
   Rating: -
   totalTime: PT25M

Pork Chops - Southern Style - Category: Pork
   Ingredients: 2       pork chops 1/2  medium    onion (diced) 1 -2 

In [13]:
f.close()